# 下載SOGO使用說明
    環境準備：
        1.在E碟新增SOGO的資料夾
        2.在SOGO的資料夾中再新增一個資料夾叫"GOOD"
        3.以後下載完的檔案都要先放到GOOD中以便讓程式去CHECK有無重復下載
    步驟：
        Step1 定義function
        stept2 main 在get_page_url中輸入想要下載的頁數，程式會拿到要下載的片名(title)與下載的網址(url)
        Step3 main2 將已經下的檔案與下載清單做比較，只下載未載過的檔案
        
        


In [81]:
#Step1 定義function
import requests as r
import json
import re
def get_page_url(page):
    url="http://oursogo.com/forum-162-"+str(page)+".html"
    head={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
         ,"Cookie":"werA_0ff6_saltkey=1I1V638W;werA_0ff6_viewadult=acceptrule;werA_0ff6_auth=39f4CCq7xHmJTR2KE8WKPgbpwrW%2FBnJEeVxdStNYAmIOfPXCVcbHbr0gePO1taYF87fJZWgVqmgUfIjv7BB4E16VfeA2;werA_0ff6_sid=un6nU6;"}
    res=r.get(url,headers=head)
    html=res.text
    
    theme_url_list=[]
    theme_url=re.findall('''http://oursogo.com/thread-2.{1,}.html''',html)
    theme_url_set=list(set(theme_url))
    #print(len(theme_url_set))
    for g in theme_url_set:
        theme_url_list.append(g)
    return theme_url_list


from bs4 import BeautifulSoup
def get_title_url(theme_url_list):
    title_url_list=[]
    for index in theme_url_list:
        head={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36","Cookie":"werA_0ff6_saltkey=1I1V638W;werA_0ff6_viewadult=acceptrule;werA_0ff6_auth=39f4CCq7xHmJTR2KE8WKPgbpwrW%2FBnJEeVxdStNYAmIOfPXCVcbHbr0gePO1taYF87fJZWgVqmgUfIjv7BB4E16VfeA2;werA_0ff6_sid=un6nU6"}
        url=index
        res=r.get(url,headers=head)
        HTML2=res.text
        soup=BeautifulSoup(HTML2,"lxml")
        title=soup.select_one("#thread_subject").text+".mp4"
        GET2=re.findall('''window\.open.{1,}按這裡下載本片''',HTML2)
        #GET2=re.findall('''https://openload.co/f/.{1,}/.{1,15}.mp4''',HTML2)
        if len(GET2)!=0:
            GETSET2=GET2[0].split("'")[1].split("\\")[0]
            title_url_list.append([title,GETSET2])
            #print(GETSET2)
            #print(title_url_list)
    return title_url_list


import splinter
from bs4 import BeautifulSoup
def get_openload_url(url):
    embed_url="https://openload.co/embed/"+url.split("/")[4]
    b = splinter.Browser('chrome')
    b.visit(embed_url)
    play = b.find_by_xpath('''//*[@id="videooverlay"]''')
    play.click()
    html=b.html
    b.quit()
    #print(html)
    soup=BeautifulSoup(html,"lxml")
    urj=soup.select_one("#DtsBlkVFQx")
    openload_url="https://openload.co/stream/"+urj.text
    print("openload "+openload_url)
    return openload_url

from bs4 import BeautifulSoup
def get_rapidvideo_url(url):
    head={"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Mobile Safari/537.36"}
    res=r.get(url,headers=head)
    html=res.text
    soup=BeautifulSoup(html,"lxml")
    rapid_url_list=soup.select(".button_small")  
    if len(rapid_url_list)==1:
        rapid_url=rapid_url_list[0]["href"]
    else:
        rapid_url=rapid_url_list[0]["href"]
    print("rapid "+rapid_url)
    return rapid_url

#下載
import requests
import shutil
def downloadvideo(url,filename):
    head={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
         }
    res=r.get(url,headers=head,stream=True)
    with open(filename,'wb') as f:
        shutil.copyfileobj(res.raw,f)
    print(filename+" DONE!!")

#抓目前已經下載的檔案名稱有哪些
import glob
def get_downloaded_list():
    list=[]
    for g in glob.glob('E:\SOGO\GOOD\*'):
        list.append(g.split("\\")[0]+"/"+g.split("\\")[1]+"/"+g.split("\\")[3])
    return list 

#去除已經下載過的檔案後再下載
from time import sleep
def download_removerepeatviedo():
    #alllist=get_chapter_url(url)
    downloadedlist=get_downloaded_list()
    need_list=[]
    for title,url in readylist:#比對想下載清單(readylist)與已下載清單(downloadedlist)差別，多的就append到need_list中
        if title not in downloadedlist:
            need_list.append([title,url])
    print("All you need to download are "+str(len(need_list)))
    print("")
    global conut
    count=1
    if len(need_list)>0:#判斷是否需要更新
        for title,url in need_list:
            print(str(count)+". You need to download: "+title)#告訴使用者需要更新的部份是
            downloadvideo(url,title)#下載新更新的部份
            count=count+1
            #print(" update done^^")
    else:
        print(" Videos which you want has been aleady downloaded")

In [65]:
#stept2 main 在get_page_url中輸入想要下載的頁數，程式會拿到要下載的片名(title)與下載的網址(url)
theme_url_list=get_page_url("1")
downloadlist=get_title_url(theme_url_list)
readylist=[]
for title,url in downloadlist:
    print("url="+url)
    if len(url.split("openload"))>1:
        print("openload_url="+url)
        url_new=get_openload_url(url)
        readylist.append(["E:/SOGO/"+title,url_new])
        print("")
    elif len(url.split("rapidvideo"))>1:
        print("rapidvideo_url="+url)
        url_new=get_rapidvideo_url(url)
        readylist.append(["E:/SOGO/"+title,url_new])
        print("")

for title,url in readylist:
    print(title)
    print(url)
    print("")
    

url=https://openload.co/f/exon98_TqDU/
openload_url=https://openload.co/f/exon98_TqDU/
openload https://openload.co/stream/exon98_TqDU~1523511540~111.248.0.0~75lKo7zU

url=https://www.rapidvideo.com/d/FQ7T5MJCZX
rapidvideo_url=https://www.rapidvideo.com/d/FQ7T5MJCZX
rapid https://www2219.playercdn.net/p-dl/0/niplcT0rqaFRaI2P4pB37A/1523597947/180410/540FQ7T6Q056XFA54UC5Z.mp4?name=041018-638-carib-1080p.mp4-360.mp4

url=https://www.rapidvideo.com/d/FQ1XFR2L6V
rapidvideo_url=https://www.rapidvideo.com/d/FQ1XFR2L6V
rapid https://www2380.playercdn.net/p-dl/0/BvxEC8zXXaCr7mEL1rbFAQ/1523597948/180406/159FQ2QUS9H5QPOXLROBR.mp4?name=MIDE-480.mp4-360.mp4

url=https://openload.co/f/5Edncf78Kc4/
openload_url=https://openload.co/f/5Edncf78Kc4/
openload https://openload.co/stream/5Edncf78Kc4~1523511554~111.248.0.0~lW36MEzg

url=https://openload.co/f/uVV41ySs2lI/
openload_url=https://openload.co/f/uVV41ySs2lI/
openload https://openload.co/stream/uVV41ySs2lI~1523511563~111.248.0.0~WXuaeHXQ

url=https:

In [ ]:
#Step 3 main2 將已經下的檔案與下載清單做比較，只下載未載過的檔案(step2 3 其實可以合併，但還在check bug中，所以未合併)
print("You find: "+str(len(readylist))+" viedos")
download_removerepeatviedo()



You find: 23 viedos
All you need to download are 20

1. You need to download: E:/SOGO/【HD】碰到姐你只有被榨乾的份！不信？試試看...逆癡漢榨到你不要不要的叫....mp4
E:/SOGO/【HD】碰到姐你只有被榨乾的份！不信？試試看...逆癡漢榨到你不要不要的叫....mp4 DONE!!
2. You need to download: E:/SOGO/加勒比 041018-638 風俗店のレクチャー係は最高の職業 うさぎ美優.mp4
E:/SOGO/加勒比 041018-638 風俗店のレクチャー係は最高の職業 うさぎ美優.mp4 DONE!!
3. You need to download: E:/SOGO/突然できた義理（爆乳）の姉と素股でヌルっと生挿入！.mp4
E:/SOGO/突然できた義理（爆乳）の姉と素股でヌルっと生挿入！.mp4 DONE!!
4. You need to download: E:/SOGO/【HD】蘿莉塔妹妹親純可愛卻想要一直被搞，被大叔集體輪插內射搞到理性崩壞的身體....mp4


In [5]:
page=1
url="http://oursogo.com/forum-162-"+str(page)+".html"
head={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
         ,"Cookie":"werA_0ff6_saltkey=1I1V638W;werA_0ff6_viewadult=acceptrule;werA_0ff6_auth=39f4CCq7xHmJTR2KE8WKPgbpwrW%2FBnJEeVxdStNYAmIOfPXCVcbHbr0gePO1taYF87fJZWgVqmgUfIjv7BB4E16VfeA2;werA_0ff6_sid=un6nU6;"}
res=r.get(url,headers=head)
html=res.text
    
theme_url_list=[]
theme_url=re.findall('''http://oursogo.com/thread-2.{1,}.html''',html)
theme_url_set=list(set(theme_url))
#print(len(theme_url_set))
for g in theme_url_set:
    theme_url_list.append(g)
    
for index in theme_url_list:
    print(index)

http://oursogo.com/thread-2527036-1-1.html
http://oursogo.com/thread-2526861-1-1.html
http://oursogo.com/thread-2526873-1-1.html
http://oursogo.com/thread-2526988-1-1.html
http://oursogo.com/thread-2526003-1-1.html
http://oursogo.com/thread-2527041-1-1.html
http://oursogo.com/thread-2526987-1-1.html
http://oursogo.com/thread-2526445-1-1.html
http://oursogo.com/thread-2526961-1-1.html
http://oursogo.com/thread-2526695-1-1.html
http://oursogo.com/thread-2526995-1-1.html
http://oursogo.com/thread-2527038-1-1.html
http://oursogo.com/thread-2527043-1-1.html
http://oursogo.com/thread-2527037-1-1.html
http://oursogo.com/thread-2526601-1-1.html
http://oursogo.com/thread-2527039-1-1.html
http://oursogo.com/thread-2526871-1-1.html
http://oursogo.com/thread-2525594-1-1.html
http://oursogo.com/thread-2506993-1-1.html
http://oursogo.com/thread-2527040-1-1.html
http://oursogo.com/thread-2527035-1-1.html
http://oursogo.com/thread-2527034-1-1.html
http://oursogo.com/thread-2526948-1-1.html
http://ours